In [1]:
from lavis.models import load_model_and_preprocess
import os
import torch
from PIL import Image
import numpy as np
from tqdm import tqdm
import sys

e:\AICHCM2024\.venv\Lib\site-packages\fairscale\experimental\nn\offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
e:\AICHCM2024\.venv\Lib\site-packages\fairscale\experimental\nn\offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [2]:
sys.path.append('..')

In [3]:
from utils.utils import start_from_index
%run ../utils/utils.py

In [4]:
from config import IMPROVED_KEYFRAME_PATH, BLIP_FEATURES_VITG_PATH

In [5]:
keyframes_dir = "../" + IMPROVED_KEYFRAME_PATH
blip_features_dir = "../" + BLIP_FEATURES_VITG_PATH

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [7]:
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_feature_extractor", model_type="pretrain", is_eval=True, device=device)

e:\AICHCM2024\.venv\Lib\site-packages\lavis\models\eva_vit.py:433: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file, map_location="cpu")
e:\

In [8]:
def process_keyframes_in_batch(folder_path, output_path, batch_size):
    video_features = []
    keyframe_paths = sorted([os.path.join(folder_path, img) for img in os.listdir(folder_path)])

    for i in range(0, len(keyframe_paths), batch_size):
        keyframes = []
        batch_paths = keyframe_paths[i:i+batch_size]
        for keyframe_path in batch_paths:
            keyframe = vis_processors["eval"](Image.open(keyframe_path)).unsqueeze(0)
            keyframes.append(keyframe)

        keyframes = torch.cat(keyframes).to(device)
        sample = {"image": keyframes, "text_input": ""}

        with torch.no_grad(), torch.amp.autocast("cuda"):
            keyframe_features = model.extract_features(sample, mode="image")
        keyframe_features = keyframe_features.image_embeds_proj[:, 0, :]
        keyframe_features /= keyframe_features.norm(dim=-1, keepdim=True)
        for idx in range(keyframe_features.shape[0]):
            video_features.append(keyframe_features[idx].detach().cpu().numpy().astype(np.float32).flatten())
    np.save(output_path, video_features)

In [9]:
keyframe_files_to_process = start_from_index(keyframes_dir, 25, "")

In [10]:
batch_size = 128
for video_folder in tqdm(keyframe_files_to_process):
    video_folder_path = os.path.join(keyframes_dir, video_folder)

    output_path = os.path.join(blip_features_dir, f"{video_folder}.npy")
    process_keyframes_in_batch(video_folder_path, output_path, batch_size)

  0%|          | 0/745 [00:00<?, ?it/s]e:\AICHCM2024\.venv\Lib\site-packages\lavis\models\blip2_models\blip2.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast(dtype=dtype)
100%|██████████| 745/745 [1:23:53<00:00,  6.76s/it]
